<a href="https://colab.research.google.com/github/ganeshred/NLPAuthorshipAttributionNLM/blob/main/Attention_bi_LSTM_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchtext==0.10.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

/usr/local/lib/python3.9/dist-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/balancetask1.csv')

In [ ]:
len(df)

4264

In [ ]:
df.head()

,Unnamed: 0,T1,T2,class
0,0,"Barbra Streisand, James Corden and More on The...",Latest Headlines on CNN Business the great shu...,1.0
1,1,Transcripts return to transcripts main page cn...,China wants to take a victory lap over its han...,1.0
2,2,China Spins Tale That the U.S. Army Started th...,Coronavirus disinformation creates challenges ...,1.0
3,3,History's deadliest air raid happened in Tokyo...,China coronavirus: Eating wild animals made il...,1.0
4,4,Surprising factors that may keep you thin cnn ...,China's economy could shrink for the first tim...,1.0


In [ ]:
df['text'] = df['T1']+df['T2']
df = df[['text','class']]

In [ ]:
df[['text']] = df[['text']]. astype(str)
df[['class']] = df[['class']].astype(int)

<ipython-input-9-96884e3488c3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['text']] = df[['text']]. astype(str)
<ipython-input-9-96884e3488c3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['class']] = df[['class']].astype(int)


In [ ]:
# text_field = data.Field(
#     sequential=True,
#     tokenize='basic_english', 
#     fix_length=500,
#     lower=True
# )
TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(sequential=False, use_vocab=False, dtype = torch.float)

# label_field = data.Field(sequential=False, use_vocab=False)

# sadly have to apply preprocess manually
preprocessed_text = df['text'].apply(
    lambda x: TEXT.preprocess(x)
)

# load fastext simple embedding with 300d
TEXT.build_vocab(
    preprocessed_text, 
    max_size = 25000,
    vectors='glove.6B.50d',
    unk_init = torch.Tensor.normal_
)

# get the vocab instance
vocab = TEXT.vocab

/usr/local/lib/python3.9/dist-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [ ]:
df.head()

,text,class
0,"Barbra Streisand, James Corden and More on The...",1
1,Transcripts return to transcripts main page cn...,1
2,China Spins Tale That the U.S. Army Started th...,1
3,History's deadliest air raid happened in Tokyo...,1
4,Surprising factors that may keep you thin cnn ...,1


In [ ]:
# ltoi = {l: i for i, l in enumerate(df['class'].unique())}

In [ ]:
df['label1'] = df['class']

class DataFrameDataset(data.Dataset):
    def __init__(self, df: pd.DataFrame, fields: list):
        super(DataFrameDataset, self).__init__(
            [
                data.Example.fromlist(list(r), fields) 
                for i, r in df.iterrows()
            ], 
            fields
        )

In [ ]:
df1=df[['text','label1']]

In [ ]:
examples=[]
for i,row in df1.iterrows():
  # print(example)
  examples.append(data.Example.fromlist(row, fields=[('text', TEXT), ('label', LABEL)]))

In [ ]:
# examples[2000].label
c=0
for i in examples:
  if type(i.label) != int:
    c+=1

In [ ]:
c

0

In [ ]:
dataset = data.Dataset(examples, fields=[('text', TEXT), ('label', LABEL)])

In [ ]:
train_dataset, test_dataset, valid_dataset = dataset.split(split_ratio=[0.6, 0.2, 0.2], stratified=True, strata_field='label')

In [ ]:
len(train_dataset)

2558

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("We are working with ", device)
# device = 'cpu'
train_iter, test_iter, valid_iter = data.BucketIterator.splits(
    datasets=(train_dataset, test_dataset, valid_dataset), 
    batch_size=32,
    sort_within_batch = True,
    sort_key=lambda x: len(x.text),
    device=device
)

We are working with  cuda


In [ ]:
import torch.nn as nn

class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.attn = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.v = nn.Linear(self.hidden_size, 1, bias=False)
        
    def forward(self, hidden, encoder_outputs):
        # hidden = [batch size, hidden size]
        # encoder_outputs = [src sent len, batch size, hidden size]
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        # print(hidden.shape)
        # repeat the last hidden state of the decoder
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)

        # print(hidden.shape)
        # print(encoder_outputs.transpose(1,0).shape)

        # # hidden = hidden.transpose(1, 0)

        # print(hidden.shape)
        
        # concatenate hidden and encoder_outputs
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs.transpose(1,0)), dim=2))) 
        
        # compute attention scores
        attention = self.v(energy).squeeze(2)
        attention = torch.softmax(attention, dim=1).unsqueeze(1)
        # compute context vector
        # print(attention.shape)
        # print(encoder_outputs.shape)
        context = torch.bmm(attention, encoder_outputs.transpose(0, 1))
        return context, attention

class LSTMAttention(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx, weight_decay=1e-5):
        super().__init__()
        #1. Initialize Embedding Layer
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=pad_idx)
        #2. Initialize LSTM layer
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=n_layers, dropout=dropout, bidirectional=bidirectional)
        #3. Initialize attention layer
        self.attention = Attention(hidden_dim*2)
        #4. Initialize a fully connected layer with Linear transformation
        self.fc = nn.Linear(hidden_dim*4, output_dim)
        #5. Initialize Dropout
        self.dropout = nn.Dropout(dropout)
        #6. Initialize a projection layer for hidden tensor
        if bidirectional:
            self.hidden_proj = nn.Linear(2*hidden_dim, 2*hidden_dim)
        else:
            self.hidden_proj = nn.Linear(hidden_dim, hidden_dim)
        self.fc.weight_decay = weight_decay
        self.fc.bias.requires_grad = False
        self.hidden_proj.weight_decay = weight_decay
        self.hidden_proj.bias.requires_grad = False
        self.attention.attn.weight_decay = weight_decay
        self.attention.v.weight_decay = weight_decay

    def forward(self, text, text_lengths):
        #1. Apply embedding layer that matches each word to its vector and apply dropout. Dim [sent_len, batch_size, emb_dim]
        embedding = self.embedding(text)
        embedding_dropout = self.dropout(embedding)
        #2. Run the LSTM along the sentences of length sent_len. #output = [sent len, batch size, hid dim * num directions]; #hidden = [num layers * num directions, batch size, hid dim]
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedding_dropout, text_lengths.to('cpu'), enforce_sorted=False)
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        finalhidden = hidden
        encoder_outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_output)
        #3. Compute the attention vector and apply attention to the encoder outputs
        if self.lstm.bidirectional:
            hidden = torch.tanh(self.hidden_proj(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)))
        else:
            hidden = torch.tanh(self.hidden_proj(hidden[-1,:,:]))
        # print(hidden.shape)
        # print(encoder_outputs.shape)
        context, attention = self.attention(hidden, encoder_outputs)
        # print(context.squeeze(1).shape)
        # finalhidden = finalhidden.unsqueeze(1)
        # print(torch.cat((finalhidden[-2,:,:], finalhidden[-1,:,:]), dim=1).shape)
        #4. Concatenate the final hidden state and the context vector
        output = torch.cat((torch.cat((finalhidden[-2,:,:], finalhidden[-1,:,:]), dim=1), context.squeeze(1)), dim=1)
        #5. Apply the fully connected layer and return the output
        output = self.dropout(output)
        return self.fc(output)



In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 50
HIDDEN_DIM = 126
OUTPUT_DIM = 2
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTMAttention(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT,
            PAD_IDX)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,004,588 trainable parameters


In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 50])


In [ ]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  0.7372,  0.3910,  0.5158],
        [ 0.1825, -0.0737,  0.3147,  ...,  1.3226, -2.6796, -0.1297],
        [ 0.4180,  0.2497, -0.4124,  ..., -0.1841, -0.1151, -0.7858],
        ...,
        [ 0.5903, -0.5771, -0.8633,  ...,  0.8456,  0.1322, -0.4744],
        [-0.0086, -0.5526,  0.5754,  ...,  0.0818,  0.8060, -0.9564],
        [ 0.7852, -0.4710, -0.1030,  ..., -1.6950, -0.1686,  0.7598]])

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.4180,  0.2497, -0.4124,  ..., -0.1841, -0.1151, -0.7858],
        ...,
        [ 0.5903, -0.5771, -0.8633,  ...,  0.8456,  0.1322, -0.4744],
        [-0.0086, -0.5526,  0.5754,  ...,  0.0818,  0.8060, -0.9564],
        [ 0.7852, -0.4710, -0.1030,  ..., -1.6950, -0.1686,  0.7598]])


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.CrossEntropyLoss()

# device='cpu'
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def multiclass_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    # get the index of the highest prediction for each instance
    _, preds = torch.max(preds, dim=1)
    correct = (preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

# def binary_accuracy(preds, y):
#     """
#     Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
#     """
#     #round predictions to the closest integer
#     rounded_preds = torch.round(torch.sigmoid(preds))
#     predicted_labels = torch.argmax(rounded_preds, dim=1)
#     correct = (predicted_labels == y).float() #convert into float for division 
#     acc = correct.sum() / len(correct)
#     return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        # print(batch.text)
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)

        target = batch.label
        target = target.type(torch.LongTensor)

        # print(target.shape)

        # print(predictions.shape)
        
        loss = criterion(predictions.to(device), target.to(device))
        
        acc = multiclass_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            
            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            target = batch.label
            target = target.type(torch.LongTensor)
            
            loss = criterion(predictions.to(device), target.to(device))
            
            acc = multiclass_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()



    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 12
path = F"/content/gdrive/My Drive/attentionbilstm_model_task1.pt"
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    train_dataset, test_dataset, valid_dataset = dataset.split(split_ratio=[0.6, 0.2, 0.2], stratified=True, strata_field='label')
    train_iter, test_iter, valid_iter = data.BucketIterator.splits(
        datasets=(train_dataset, test_dataset, valid_dataset), 
        batch_size=32,
        sort_within_batch = True,
        sort_key=lambda x: len(x.text),
        device=device
    )
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), path)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 26s
	Train Loss: 0.702 | Train Acc: 50.19%
	 Val. Loss: 0.694 |  Val. Acc: 48.31%
Epoch: 02 | Epoch Time: 0m 25s
	Train Loss: 0.696 | Train Acc: 49.95%
	 Val. Loss: 0.694 |  Val. Acc: 47.69%
Epoch: 03 | Epoch Time: 0m 25s
	Train Loss: 0.694 | Train Acc: 50.54%
	 Val. Loss: 0.696 |  Val. Acc: 45.83%
Epoch: 04 | Epoch Time: 0m 26s
	Train Loss: 0.691 | Train Acc: 53.36%
	 Val. Loss: 0.704 |  Val. Acc: 46.00%
Epoch: 05 | Epoch Time: 0m 24s
	Train Loss: 0.688 | Train Acc: 54.25%
	 Val. Loss: 0.717 |  Val. Acc: 44.42%
Epoch: 06 | Epoch Time: 0m 26s
	Train Loss: 0.681 | Train Acc: 57.43%
	 Val. Loss: 0.728 |  Val. Acc: 43.17%
Epoch: 07 | Epoch Time: 0m 25s
	Train Loss: 0.662 | Train Acc: 61.54%
	 Val. Loss: 0.772 |  Val. Acc: 40.30%
Epoch: 08 | Epoch Time: 0m 25s
	Train Loss: 0.638 | Train Acc: 63.03%
	 Val. Loss: 0.803 |  Val. Acc: 38.52%
Epoch: 09 | Epoch Time: 0m 25s
	Train Loss: 0.611 | Train Acc: 66.07%
	 Val. Loss: 0.957 |  Val. Acc: 37.52%
Epoch: 10 | Epoch T

In [ ]:
test_loss, test_acc = evaluate(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.077 | Test Acc: 35.08%
